# 📊 PyPortfolio - Otimização de Portfólio v2.4

**Markowitz, Fronteira Eficiente, Monte Carlo e Sharpe vs CDI**

---

### Changelog v2.4
- ✅ **Detecção automática de arquivos Excel** no diretório (com escolha interativa)
- ✅ **Leitura robusta da coluna de pesos** (aceita variações: "Carteria", "Carteira", etc.)
- ✅ **Remoção do USDC** do universo de otimização (tratado como caixa separada)
- ✅ **Curva da Fronteira Eficiente** calculada por otimização (não só pontos)
- ✅ **Monte Carlo melhorado** com cantos, sparse sampling e melhor cobertura
- ✅ **VOL_FLOOR aplicado na matriz de covariância** (evita vol~0 artificial)
- ✅ **Relatório de qualidade de dados** detalhado
- ✅ Fuzzy matching melhorado para nomes de ativos (hífen/barra tolerante)

### Problemas corrigidos do v2.3:
- GMV com vol≈0% (causado pelo USDC)
- Sharpe de -145 no GMV
- Fronteira inexistente no gráfico
- 28% dos pesos não mapeados (nomes truncados/diferentes)

---
## 🔧 BLOCO 0 - CONFIGURAÇÕES GLOBAIS

In [16]:
# =============================================================================
# CONFIGURAÇÕES GLOBAIS - EDITE AQUI
# =============================================================================

# --- Caminhos de Arquivos ---
EXCEL_DIR = "1 - Dados/1 - Rentabilidade atual"
EXCEL_PATTERN = "Rendimentos_Mensais_Ativos*.xlsx"  # Padrão para busca

# --- Período de Análise ---
DATA_INICIO = "2020-01-01"
DATA_FIM = "2026-01-01"

# --- Parâmetros de Qualidade de Dados ---
MIN_OBS = 12              # Mínimo de meses por ativo
MIN_OVERLAP = 12          # Mínimo de meses em comum para covariância
VOL_FLOOR_MENSAL = 0.005  # 0.5% ao mês (~1.7% a.a.) - piso para evitar vol~0

# --- Ativos a excluir do universo ---
ATIVOS_EXCLUIR = ["USDC"]  # Stablecoins = caixa (fora da otimização)

# --- Taxa Livre de Risco ---
RF_MODO = "atual"  # "atual" | "media_periodo" | "manual"
RF_MANUAL = 0.1150         # 11.5% a.a. (fallback)
RF_FALLBACK = 0.10

# --- Parâmetros de Otimização ---
PESO_MAX_ATIVO = 1.0
PESO_MIN_ATIVO = 0.0

# --- Monte Carlo ---
NUM_PORTFOLIOS = 80000
RANDOM_SEED = 42
MC_ALPHA_CONC = 0.2       # Dirichlet concentrado
MC_ALPHA_DIV = 1.0        # Dirichlet diversificado
MC_FRAC_CANTOS = 0.05     # 5% carteiras quase-100% em 1 ativo
MC_FRAC_SPARSE = 0.15     # 15% carteiras sparse (3-8 ativos)

# --- Fronteira Eficiente ---
N_PONTOS_FRONTEIRA = 60   # Pontos na curva

# --- Retorno ---
USAR_MEDIA_GEOMETRICA = True

# --- Visualização ---
TOP_N_PESOS = 15

print("✅ Configurações carregadas!")
print(f"   Diretório Excel: {EXCEL_DIR}")
print(f"   MIN_OBS: {MIN_OBS} meses | VOL_FLOOR: {VOL_FLOOR_MENSAL*100:.1f}% mensal")
print(f"   Ativos excluídos: {ATIVOS_EXCLUIR}")

✅ Configurações carregadas!
   Diretório Excel: 1 - Dados/1 - Rentabilidade atual
   MIN_OBS: 12 meses | VOL_FLOOR: 0.5% mensal
   Ativos excluídos: ['USDC']


---
## 📦 BLOCO 1 - BIBLIOTECAS E FUNÇÕES UTILITÁRIAS

In [2]:
# =============================================================================
# IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import eigh
import plotly.graph_objects as go
import warnings
from pathlib import Path
from difflib import SequenceMatcher
import re
from typing import Optional, Dict, Tuple, List
from IPython.display import display
import glob

try:
    import yfinance as yf
    YFINANCE_DISPONIVEL = True
except ImportError:
    YFINANCE_DISPONIVEL = False

warnings.filterwarnings('ignore', category=FutureWarning)
pd.set_option('display.max_columns', 20)
print("✅ Bibliotecas importadas!")

✅ Bibliotecas importadas!


In [3]:
# =============================================================================
# FUNÇÕES - BUSCA E SELEÇÃO DE ARQUIVO EXCEL
# =============================================================================

def encontrar_arquivos_excel(diretorio: str, pattern: str = "*.xlsx") -> List[Path]:
    """Encontra arquivos Excel no diretório que correspondem ao padrão."""
    dir_path = Path(diretorio)
    if not dir_path.exists():
        return []
    
    arquivos = list(dir_path.glob(pattern))
    # Ordenar por data de modificação (mais recente primeiro)
    arquivos.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    return arquivos

def selecionar_arquivo_excel(diretorio: str, pattern: str, caminho_padrao: str = None) -> str:
    """
    Seleciona arquivo Excel com interação do usuário.
    Se caminho_padrao existir, usa direto. Senão, busca e pergunta.
    """
    # Tentar caminho padrão primeiro
    if caminho_padrao and Path(caminho_padrao).exists():
        print(f"✅ Arquivo encontrado: {caminho_padrao}")
        return caminho_padrao
    
    # Buscar arquivos no diretório
    arquivos = encontrar_arquivos_excel(diretorio, pattern)
    
    if not arquivos:
        raise FileNotFoundError(f"Nenhum arquivo Excel encontrado em {diretorio} com padrão {pattern}")
    
    if len(arquivos) == 1:
        print(f"✅ Único arquivo encontrado: {arquivos[0].name}")
        return str(arquivos[0])
    
    # Múltiplos arquivos - mostrar opções
    print(f"\n📂 Encontrados {len(arquivos)} arquivos Excel:")
    for i, arq in enumerate(arquivos):
        mtime = pd.Timestamp.fromtimestamp(arq.stat().st_mtime).strftime("%Y-%m-%d %H:%M")
        print(f"   [{i}] {arq.name} (modificado: {mtime})")
    
    print(f"\n   [ENTER] = usar mais recente ({arquivos[0].name})")
    
    # Em ambiente não-interativo, usar o mais recente
    try:
        escolha = input("Escolha (número ou ENTER): ").strip()
        if escolha == "":
            idx = 0
        else:
            idx = int(escolha)
    except:
        idx = 0
        print("   (modo não-interativo: usando mais recente)")
    
    if 0 <= idx < len(arquivos):
        print(f"✅ Selecionado: {arquivos[idx].name}")
        return str(arquivos[idx])
    else:
        print(f"⚠️ Índice inválido, usando mais recente")
        return str(arquivos[0])

print("✅ Funções de seleção de arquivo carregadas!")

✅ Funções de seleção de arquivo carregadas!


In [4]:
# =============================================================================
# FUNÇÕES - LEITURA E PROCESSAMENTO DE DADOS
# =============================================================================

def normalizar_nome(nome: str) -> str:
    """Normaliza nome de ativo para comparação."""
    if pd.isna(nome):
        return ""
    nome = str(nome).lower().strip()
    # Acentos
    acentos = {'á':'a','à':'a','ã':'a','â':'a','é':'e','ê':'e','í':'i',
               'ó':'o','ô':'o','õ':'o','ú':'u','ç':'c'}
    for k, v in acentos.items():
        nome = nome.replace(k, v)
    # Normalizar separadores (hífen, barra, underline -> espaço)
    nome = re.sub(r'[-/_]', ' ', nome)
    # Remover caracteres especiais
    nome = re.sub(r'[^a-z0-9\s]', ' ', nome)
    nome = re.sub(r'\s+', ' ', nome).strip()
    return nome

def similaridade(a: str, b: str) -> float:
    """Retorna score de similaridade entre 0 e 1."""
    na, nb = normalizar_nome(a), normalizar_nome(b)
    # Score base
    score = SequenceMatcher(None, na, nb).ratio()
    
    # Bonus se um contém o outro (prefixo/sufixo)
    if na in nb or nb in na:
        score = max(score, 0.85)
    
    # Bonus para matches parciais de palavras-chave
    words_a = set(na.split())
    words_b = set(nb.split())
    common = words_a & words_b
    if len(common) >= 2:
        score = max(score, 0.75 + 0.05 * len(common))
    
    return min(score, 1.0)

def parse_percentual(valor) -> float:
    """Converte valor percentual para float decimal."""
    if pd.isna(valor) or valor == '-' or valor == '':
        return np.nan
    if isinstance(valor, (int, float)):
        if abs(valor) > 1:
            return valor / 100
        return valor
    valor = str(valor).strip().replace('%', '').replace(',', '.')
    try:
        return float(valor) / 100
    except:
        return np.nan

def ler_retornos_aba_excel(xlsx_path: str, sheet_name: str) -> pd.Series:
    """Lê retornos mensais de uma aba do Excel."""
    df = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None, engine='openpyxl')
    
    header_row = None
    for i, row in df.iterrows():
        if str(row.iloc[0]).strip().lower() == 'ano':
            header_row = i
            break
    
    if header_row is None:
        raise ValueError(f"Cabeçalho 'Ano' não encontrado na aba '{sheet_name}'")
    
    meses_map = {'jan':1,'fev':2,'mar':3,'abr':4,'mai':5,'jun':6,
                 'jul':7,'ago':8,'set':9,'out':10,'nov':11,'dez':12}
    
    header = [str(c).strip().lower()[:3] for c in df.iloc[header_row].values]
    
    retornos = []
    for i in range(header_row + 1, len(df)):
        row = df.iloc[i]
        ano_val = row.iloc[0]
        
        if pd.isna(ano_val) or str(ano_val).strip().lower() in ['', 'estatísticas', 'estatisticas', 'meses']:
            break
        
        try:
            ano = int(ano_val)
        except:
            break
        
        for col_idx, col_name in enumerate(header[1:], start=1):
            if col_name in meses_map:
                mes = meses_map[col_name]
                val = parse_percentual(row.iloc[col_idx])
                if not pd.isna(val):
                    data = pd.Timestamp(year=ano, month=mes, day=1)
                    retornos.append({'data': data, 'retorno': val})
    
    if not retornos:
        return pd.Series(dtype=float)
    
    df_ret = pd.DataFrame(retornos).set_index('data').sort_index()
    return df_ret['retorno']

def ler_pesos_resumo_robusto(xlsx_path: str) -> Tuple[Dict[str, float], str]:
    """
    Lê pesos da aba 'Resumo' com busca robusta da coluna de pesos.
    Aceita variações: "Peso Carteira", "Peso Carteria", "Peso Cart", etc.
    
    Returns:
        pesos: Dict[ativo, peso]
        status: "encontrado" | "nao_encontrado" | "erro"
    """
    try:
        df = pd.read_excel(xlsx_path, sheet_name='Resumo', header=None, engine='openpyxl')
    except Exception as e:
        return {}, f"erro: {str(e)[:50]}"
    
    # Buscar coluna de pesos com palavras-chave flexíveis
    header_row = None
    col_ativo = None
    col_peso = None
    
    keywords_peso = ['peso', 'weight', 'alocacao', 'alocação']
    keywords_carteira = ['cart', 'portfolio', 'atual', 'current']
    
    for i, row in df.iterrows():
        for j, val in enumerate(row):
            if pd.isna(val):
                continue
            val_str = str(val).strip().lower()
            
            # Coluna de ativo
            if val_str == 'ativo' or val_str == 'asset':
                col_ativo = j
            
            # Coluna de peso (busca flexível)
            has_peso = any(kw in val_str for kw in keywords_peso)
            has_cart = any(kw in val_str for kw in keywords_carteira)
            if has_peso and has_cart:
                col_peso = j
        
        if col_ativo is not None and col_peso is not None:
            header_row = i
            break
    
    # Relatório de busca
    if col_peso is None:
        # Tentar encontrar qualquer coluna com "peso"
        for i, row in df.iterrows():
            for j, val in enumerate(row):
                if pd.isna(val):
                    continue
                val_str = str(val).strip().lower()
                if 'peso' in val_str:
                    print(f"   ⚠️ Coluna encontrada mas não reconhecida: '{val}' (col {j})")
        return {}, "nao_encontrado"
    
    print(f"   ✅ Coluna de pesos encontrada: '{df.iloc[header_row, col_peso]}' (linha {header_row}, col {col_peso})")
    
    # Extrair pesos
    pesos = {}
    for i in range(header_row + 1, len(df)):
        row = df.iloc[i]
        ativo = row.iloc[col_ativo]
        peso = row.iloc[col_peso]
        
        if pd.isna(ativo) or str(ativo).strip() == '':
            continue
        
        ativo = str(ativo).strip()
        try:
            peso_val = float(peso) if not pd.isna(peso) else 0.0
            if peso_val > 1:  # Se > 1, provavelmente está em %
                peso_val = peso_val / 100
            pesos[ativo] = peso_val
        except:
            pesos[ativo] = 0.0
    
    return pesos, "encontrado"

def carregar_dados_excel(xlsx_path: str, min_obs: int = 12, ativos_excluir: List[str] = None):
    """
    Carrega retornos mensais e pesos do Excel.
    Exclui ativos especificados (ex: USDC).
    """
    if ativos_excluir is None:
        ativos_excluir = []
    
    xlsx = pd.ExcelFile(xlsx_path, engine='openpyxl')
    abas = xlsx.sheet_names
    
    relatorio = {
        'total_abas': len(abas),
        'abas_processadas': [],
        'abas_excluidas': [],
        'meses_por_ativo': {},
        'ativos_excluidos_manual': []
    }
    
    # Ler pesos
    print("\n📊 Buscando coluna de pesos...")
    pesos_carteira, status_pesos = ler_pesos_resumo_robusto(xlsx_path)
    relatorio['status_pesos'] = status_pesos
    
    if status_pesos == "nao_encontrado":
        print("   ⚠️ ATENÇÃO: Coluna de pesos não encontrada! Verifique o nome da coluna no Excel.")
        print("   Esperado: 'Peso Carteira Atual' ou similar")
    
    # Ler retornos de cada aba
    series_list = {}
    for aba in abas:
        if aba.lower() == 'resumo':
            continue
        
        # Verificar exclusão manual
        if aba.upper() in [a.upper() for a in ativos_excluir]:
            relatorio['ativos_excluidos_manual'].append(aba)
            continue
        
        try:
            serie = ler_retornos_aba_excel(xlsx_path, aba)
            n_obs = len(serie.dropna())
            relatorio['meses_por_ativo'][aba] = n_obs
            
            if n_obs >= min_obs:
                series_list[aba] = serie
                relatorio['abas_processadas'].append(aba)
            else:
                relatorio['abas_excluidas'].append((aba, f'{n_obs} meses < {min_obs}'))
        except Exception as e:
            relatorio['abas_excluidas'].append((aba, str(e)[:50]))
    
    if not series_list:
        raise ValueError("Nenhum ativo com dados suficientes!")
    
    retornos_df = pd.DataFrame(series_list)
    return retornos_df, pesos_carteira, relatorio

print("✅ Funções de leitura carregadas!")

✅ Funções de leitura carregadas!


In [5]:
# =============================================================================
# FUNÇÕES - ESTATÍSTICAS E MATRIZ DE COVARIÂNCIA
# =============================================================================

def retorno_geometrico_anual(retornos_mensais: pd.Series) -> float:
    """Calcula retorno geométrico anualizado."""
    ret = retornos_mensais.dropna()
    if len(ret) == 0:
        return np.nan
    prod = np.prod(1 + ret)
    n = len(ret)
    return float(prod ** (12 / n) - 1)

def retorno_aritmetico_anual(retornos_mensais: pd.Series) -> float:
    """Calcula retorno aritmético anualizado."""
    return float(retornos_mensais.mean() * 12)

def volatilidade_anual(retornos_mensais: pd.Series) -> float:
    """Calcula volatilidade anualizada."""
    return float(retornos_mensais.std() * np.sqrt(12))

def aplicar_vol_floor_covariancia(cov_matrix: pd.DataFrame, vol_floor_mensal: float) -> pd.DataFrame:
    """
    Aplica piso de volatilidade na DIAGONAL da matriz de covariância.
    Evita que ativos com vol~0 dominem a otimização.
    
    vol_floor_mensal: volatilidade mensal mínima (ex: 0.005 = 0.5%)
    """
    cov_floor_mensal = vol_floor_mensal ** 2  # Variância
    cov_floor_anual = cov_floor_mensal * 12   # Anualizada
    
    cov_ajustada = cov_matrix.copy()
    ativos_ajustados = []
    
    for i, ativo in enumerate(cov_matrix.index):
        var_atual = cov_matrix.iloc[i, i]
        if var_atual < cov_floor_anual:
            cov_ajustada.iloc[i, i] = cov_floor_anual
            ativos_ajustados.append(ativo)
    
    if ativos_ajustados:
        print(f"   ⚠️ VOL_FLOOR aplicado em {len(ativos_ajustados)} ativos: {ativos_ajustados[:5]}{'...' if len(ativos_ajustados) > 5 else ''}")
    
    return cov_ajustada

def corrigir_matriz_psd(cov_matrix: pd.DataFrame, epsilon: float = 1e-8) -> Tuple[pd.DataFrame, bool]:
    """Verifica se matriz é PSD e corrige se necessário."""
    arr = cov_matrix.values
    eigenvalues, eigenvectors = eigh(arr)
    
    if np.min(eigenvalues) >= -epsilon:
        return cov_matrix, False
    
    eigenvalues_corrigidos = np.maximum(eigenvalues, epsilon)
    arr_corrigido = eigenvectors @ np.diag(eigenvalues_corrigidos) @ eigenvectors.T
    arr_corrigido = (arr_corrigido + arr_corrigido.T) / 2
    
    cov_corrigida = pd.DataFrame(arr_corrigido, index=cov_matrix.index, columns=cov_matrix.columns)
    return cov_corrigida, True

def calcular_estatisticas_portfolio(retornos_df: pd.DataFrame, usar_geometrico: bool = True, 
                                   vol_floor_mensal: float = 0.0) -> Tuple[pd.Series, pd.DataFrame]:
    """Calcula retornos esperados e matriz de covariância anualizados."""
    if usar_geometrico:
        retornos_esperados = retornos_df.apply(retorno_geometrico_anual)
    else:
        retornos_esperados = retornos_df.apply(retorno_aritmetico_anual)
    
    # Covariância anualizada
    cov_matrix = retornos_df.cov() * 12
    
    # Aplicar piso de volatilidade
    if vol_floor_mensal > 0:
        cov_matrix = aplicar_vol_floor_covariancia(cov_matrix, vol_floor_mensal)
    
    return retornos_esperados, cov_matrix

print("✅ Funções de estatística carregadas!")

✅ Funções de estatística carregadas!


In [6]:
# =============================================================================
# FUNÇÕES - OTIMIZAÇÃO
# =============================================================================

def calcular_metricas_portfolio(pesos, retornos_esperados, cov_matrix, rf_anual=0.0):
    """Calcula métricas de um portfólio."""
    pesos = np.array(pesos, dtype=float)
    retorno = float(np.dot(pesos, retornos_esperados))
    volatilidade = float(np.sqrt(np.dot(pesos.T, np.dot(cov_matrix, pesos))))
    
    if volatilidade > 1e-8:
        sharpe = (retorno - rf_anual) / volatilidade
    else:
        sharpe = np.nan
    
    return retorno, volatilidade, sharpe

def otimizar_min_volatilidade(retornos_esperados, cov_matrix, peso_min=0.0, peso_max=1.0):
    """Encontra carteira GMV (Global Minimum Variance)."""
    n = len(retornos_esperados)
    pesos_iniciais = np.ones(n) / n
    limites = tuple((peso_min, peso_max) for _ in range(n))
    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}
    
    def objetivo(w):
        return np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))
    
    result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites, 
                     constraints=restricoes, options={'maxiter': 1000})
    return result.x

def otimizar_max_sharpe(retornos_esperados, cov_matrix, rf_anual=0.0, peso_min=0.0, peso_max=1.0):
    """Encontra carteira de máximo Sharpe."""
    n = len(retornos_esperados)
    pesos_iniciais = np.ones(n) / n
    limites = tuple((peso_min, peso_max) for _ in range(n))
    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}
    
    def objetivo(w):
        ret = np.dot(w, retornos_esperados)
        vol = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))
        if vol < 1e-8:
            return 1e10
        return -(ret - rf_anual) / vol
    
    result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites, 
                     constraints=restricoes, options={'maxiter': 1000})
    return result.x

def otimizar_para_retorno_alvo(retornos_esperados, cov_matrix, retorno_alvo, peso_min=0.0, peso_max=1.0):
    """Encontra carteira de mínima variância para um retorno-alvo."""
    n = len(retornos_esperados)
    pesos_iniciais = np.ones(n) / n
    limites = tuple((peso_min, peso_max) for _ in range(n))
    restricoes = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
        {'type': 'eq', 'fun': lambda x: np.dot(x, retornos_esperados) - retorno_alvo}
    ]
    
    def objetivo(w):
        return np.dot(w.T, np.dot(cov_matrix, w))
    
    result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites, 
                     constraints=restricoes, options={'maxiter': 500})
    
    if result.success:
        return result.x
    return None

def calcular_fronteira_eficiente(retornos_esperados, cov_matrix, rf_anual, n_pontos=50, 
                                 peso_min=0.0, peso_max=1.0):
    """
    Calcula a fronteira eficiente por otimização.
    Retorna pontos (vol, ret) da curva.
    """
    # Primeiro encontrar GMV e Max Sharpe como limites
    pesos_gmv = otimizar_min_volatilidade(retornos_esperados, cov_matrix, peso_min, peso_max)
    ret_gmv, vol_gmv, _ = calcular_metricas_portfolio(pesos_gmv, retornos_esperados, cov_matrix, rf_anual)
    
    pesos_max_sharpe = otimizar_max_sharpe(retornos_esperados, cov_matrix, rf_anual, peso_min, peso_max)
    ret_max_sharpe, vol_max_sharpe, _ = calcular_metricas_portfolio(pesos_max_sharpe, retornos_esperados, cov_matrix, rf_anual)
    
    # Retorno máximo possível (100% no melhor ativo)
    ret_max = np.max(retornos_esperados)
    
    # Gerar grade de retornos-alvo (do GMV ao máximo)
    retornos_alvo = np.linspace(ret_gmv, min(ret_max, ret_max_sharpe * 1.5), n_pontos)
    
    fronteira_vols = []
    fronteira_rets = []
    
    for ret_alvo in retornos_alvo:
        pesos = otimizar_para_retorno_alvo(retornos_esperados, cov_matrix, ret_alvo, peso_min, peso_max)
        if pesos is not None:
            ret, vol, _ = calcular_metricas_portfolio(pesos, retornos_esperados, cov_matrix, rf_anual)
            fronteira_vols.append(vol)
            fronteira_rets.append(ret)
    
    # Remover duplicatas e ordenar
    pontos = list(zip(fronteira_vols, fronteira_rets))
    pontos = sorted(set(pontos), key=lambda x: x[0])  # Ordenar por vol
    
    if pontos:
        fronteira_vols, fronteira_rets = zip(*pontos)
        return np.array(fronteira_vols), np.array(fronteira_rets)
    else:
        return np.array([vol_gmv]), np.array([ret_gmv])

print("✅ Funções de otimização carregadas!")

✅ Funções de otimização carregadas!


In [7]:
# =============================================================================
# FUNÇÕES - MONTE CARLO (MELHORADO)
# =============================================================================

def simular_portfolios_monte_carlo_v2(
    retornos_esperados: np.ndarray, 
    cov_matrix: np.ndarray, 
    rf_anual: float,
    n_portfolios: int = 50000,
    seed: int = 42,
    alpha_conc: float = 0.2,
    alpha_div: float = 1.0,
    frac_cantos: float = 0.05,
    frac_sparse: float = 0.15
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Monte Carlo melhorado com:
    - Cantos (quase 100% em 1 ativo)
    - Sparse sampling (poucos ativos por vez)
    - Mix concentrado/diversificado
    """
    np.random.seed(seed)
    n_ativos = len(retornos_esperados)
    
    n_cantos = int(n_portfolios * frac_cantos)
    n_sparse = int(n_portfolios * frac_sparse)
    n_conc = int((n_portfolios - n_cantos - n_sparse) * 0.6)
    n_div = n_portfolios - n_cantos - n_sparse - n_conc
    
    pesos_list = []
    
    # 1. Cantos (quase 100% em 1 ativo)
    for _ in range(n_cantos):
        w = np.zeros(n_ativos)
        idx_principal = np.random.randint(0, n_ativos)
        peso_principal = np.random.uniform(0.85, 0.98)
        w[idx_principal] = peso_principal
        # Distribuir resto aleatoriamente
        resto = 1.0 - peso_principal
        outros_idx = [i for i in range(n_ativos) if i != idx_principal]
        if outros_idx:
            w_resto = np.random.dirichlet([0.5] * len(outros_idx)) * resto
            for i, idx in enumerate(outros_idx):
                w[idx] = w_resto[i]
        pesos_list.append(w)
    
    # 2. Sparse (3-8 ativos)
    for _ in range(n_sparse):
        w = np.zeros(n_ativos)
        k = np.random.randint(3, min(9, n_ativos + 1))
        idx_selecionados = np.random.choice(n_ativos, size=k, replace=False)
        w_k = np.random.dirichlet([0.5] * k)
        w[idx_selecionados] = w_k
        pesos_list.append(w)
    
    # 3. Concentrado (Dirichlet com alpha baixo)
    w_conc = np.random.dirichlet([alpha_conc] * n_ativos, size=n_conc)
    pesos_list.extend(w_conc)
    
    # 4. Diversificado (Dirichlet com alpha=1)
    w_div = np.random.dirichlet([alpha_div] * n_ativos, size=n_div)
    pesos_list.extend(w_div)
    
    pesos = np.array(pesos_list)
    
    # Calcular métricas (vetorizado)
    retornos = pesos @ retornos_esperados
    volatilidades = np.sqrt(np.einsum('ij,jk,ik->i', pesos, cov_matrix, pesos))
    sharpes = np.where(volatilidades > 1e-8, (retornos - rf_anual) / volatilidades, np.nan)
    
    return retornos, volatilidades, sharpes, pesos

print("✅ Funções de Monte Carlo carregadas!")

✅ Funções de Monte Carlo carregadas!


In [8]:
# =============================================================================
# FUNÇÕES - TAXA LIVRE DE RISCO (CDI)
# =============================================================================

def obter_cdi_atual():
    """Obtém CDI atual via API do BCB."""
    try:
        url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados/ultimos/1?formato=json"
        df = pd.read_json(url)
        if df.empty:
            raise ValueError("SGS retornou vazio")
        valor_str = str(df.loc[0, "valor"]).replace(",", ".")
        taxa_diaria = float(valor_str) / 100.0
        return float((1.0 + taxa_diaria) ** 252 - 1.0)
    except Exception as e:
        print(f"⚠️ Erro ao obter CDI: {e}")
        return None

def obter_cdi_periodo(data_inicio: str, data_fim: str):
    """Obtém CDI médio do período."""
    try:
        di = pd.to_datetime(data_inicio).strftime("%d/%m/%Y")
        df_str = pd.to_datetime(data_fim).strftime("%d/%m/%Y")
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json&dataInicial={di}&dataFinal={df_str}"
        
        cdi = pd.read_json(url)
        if cdi.empty:
            return None
        
        cdi["valor"] = cdi["valor"].astype(str).str.replace(",", ".", regex=False)
        cdi["valor"] = pd.to_numeric(cdi["valor"], errors="coerce") / 100.0
        
        # Média geométrica anualizada
        return float(np.expm1(np.log1p(cdi["valor"].dropna()).mean() * 252))
    except Exception as e:
        print(f"⚠️ Erro ao obter CDI período: {e}")
        return None

def obter_taxa_livre_risco(modo, fallback, manual=None, data_inicio=None, data_fim=None):
    """Obtém taxa livre de risco conforme modo."""
    if modo == "manual":
        return manual if manual is not None else fallback
    
    if modo == "atual":
        rf = obter_cdi_atual()
        return rf if rf is not None else fallback
    
    if modo == "media_periodo" and data_inicio and data_fim:
        rf = obter_cdi_periodo(data_inicio, data_fim)
        if rf is not None:
            return rf
        # Fallback para atual
        rf = obter_cdi_atual()
        return rf if rf is not None else fallback
    
    return fallback

print("✅ Funções de RF carregadas!")

✅ Funções de RF carregadas!


In [9]:
# =============================================================================
# FUNÇÕES - VISUALIZAÇÃO
# =============================================================================

def criar_grafico_fronteira_completo(
    ret_sim, vol_sim, sharpe_sim,
    fronteira_vols, fronteira_rets,
    carteiras_especiais, rf_anual,
    titulo="Fronteira Eficiente"
):
    """Cria gráfico com nuvem Monte Carlo + curva da fronteira."""
    fig = go.Figure()
    
    # 1. Nuvem de portfólios
    fig.add_trace(go.Scatter(
        x=vol_sim * 100, y=ret_sim * 100,
        mode='markers',
        marker=dict(size=2, color=sharpe_sim, colorscale='Viridis',
                    colorbar=dict(title='Sharpe', x=1.02), opacity=0.4),
        name='Portfólios Simulados',
        hovertemplate='Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<br>Sharpe: %{marker.color:.3f}<extra></extra>'
    ))
    
    # 2. Curva da fronteira eficiente
    fig.add_trace(go.Scatter(
        x=fronteira_vols * 100, y=fronteira_rets * 100,
        mode='lines',
        line=dict(color='red', width=3),
        name='Fronteira Eficiente',
        hovertemplate='Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>'
    ))
    
    # 3. Carteiras especiais
    estilos = {
        'GMV': {'color': 'blue', 'symbol': 'diamond', 'size': 18},
        'Max Sharpe': {'color': 'green', 'symbol': 'triangle-up', 'size': 18},
        'Carteira Atual': {'color': 'gold', 'symbol': 'star', 'size': 24}
    }
    
    for nome, dados in carteiras_especiais.items():
        estilo = estilos.get(nome, {'color': 'purple', 'symbol': 'circle', 'size': 15})
        
        pesos = dados.get('pesos', [])
        nomes_ativos = dados.get('nomes', [])
        if len(pesos) > 0 and len(nomes_ativos) > 0:
            idx_sorted = np.argsort(pesos)[::-1]
            top_pesos = '<br>'.join([f"{nomes_ativos[i]}: {pesos[i]*100:.1f}%" 
                                     for i in idx_sorted[:10] if pesos[i] > 0.01])
        else:
            top_pesos = ''
        
        sharpe_str = f"{dados['sharpe']:.3f}" if not np.isnan(dados['sharpe']) else "N/A"
        hover_text = (f"<b>{nome}</b><br>"
                      f"Retorno: {dados['retorno']*100:.2f}%<br>"
                      f"Volatilidade: {dados['volatilidade']*100:.2f}%<br>"
                      f"Sharpe: {sharpe_str}<br><br>"
                      f"<b>Top Pesos:</b><br>{top_pesos}")
        
        fig.add_trace(go.Scatter(
            x=[dados['volatilidade'] * 100], y=[dados['retorno'] * 100],
            mode='markers+text',
            marker=dict(size=estilo['size'], color=estilo['color'], symbol=estilo['symbol'],
                        line=dict(width=2, color='black')),
            text=[nome], textposition='top center', name=nome,
            hovertemplate=hover_text + '<extra></extra>'
        ))
    
    # Linha de RF
    fig.add_hline(y=rf_anual * 100, line_dash="dash", line_color="gray",
                  annotation_text=f"RF (CDI): {rf_anual*100:.2f}%")
    
    # Nota sobre Sharpe negativo
    if rf_anual > 0.10:
        fig.add_annotation(
            x=0.02, y=0.02, xref="paper", yref="paper",
            text=f"⚠️ RF alto ({rf_anual*100:.1f}%): Sharpe negativo é esperado para ativos com retorno < CDI",
            showarrow=False, font=dict(size=10, color="gray"),
            align="left"
        )
    
    fig.update_layout(
        title=dict(text=titulo, x=0.5),
        xaxis_title="Volatilidade Anual (%)",
        yaxis_title="Retorno Anual Esperado (%)",
        template="plotly_white", width=1100, height=750,
        legend=dict(x=0.02, y=0.98), hovermode='closest'
    )
    
    return fig

def criar_heatmap_correlacao(corr_matrix, titulo="Matriz de Correlação"):
    """Cria heatmap da matriz de correlação."""
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=list(corr_matrix.columns), y=list(corr_matrix.index),
        colorscale='RdBu', zmid=0, zmin=-1, zmax=1,
        colorbar=dict(title='Correlação'),
        hovertemplate='%{x} vs %{y}<br>Corr: %{z:.3f}<extra></extra>'
    ))
    
    fig.update_layout(title=dict(text=titulo, x=0.5), template="plotly_white",
                      width=900, height=800, xaxis=dict(side='top'))
    return fig

def criar_grafico_barras_pesos(carteiras: Dict, nomes_ativos: List[str], top_n: int = 15):
    """Cria gráfico de barras comparando pesos das carteiras."""
    fig = go.Figure()
    
    cores = {'GMV': '#636EFA', 'Max Sharpe': '#00CC96', 'Carteira Atual': '#FFA15A'}
    
    for nome_cart, dados in carteiras.items():
        pesos = np.array(dados.get('pesos', []))
        if len(pesos) == 0:
            continue
        
        # Pegar top N ativos (por peso máximo entre carteiras)
        df_temp = pd.DataFrame({'ativo': nomes_ativos, 'peso': pesos})
        df_temp = df_temp.nlargest(top_n, 'peso')
        
        fig.add_trace(go.Bar(
            name=nome_cart,
            x=df_temp['ativo'],
            y=df_temp['peso'] * 100,
            marker_color=cores.get(nome_cart, '#AB63FA')
        ))
    
    fig.update_layout(
        title="Comparativo de Pesos - Top Ativos",
        xaxis_title="Ativo",
        yaxis_title="Peso (%)",
        barmode='group',
        template="plotly_white",
        width=1000, height=500
    )
    
    return fig

print("✅ Funções de visualização carregadas!")

✅ Funções de visualização carregadas!


---
## 📥 BLOCO 2 - CARREGAMENTO DE DADOS

In [11]:
# =============================================================================
# SELEÇÃO E CARREGAMENTO DO ARQUIVO EXCEL
# =============================================================================

print("=" * 60)
print("SELEÇÃO DE FONTE DE DADOS")
print("=" * 60)

# Construir caminho padrão
caminho_padrao = f"{EXCEL_DIR}/{EXCEL_PATTERN.replace('*', 'v7.0')}"
caminho_padrao = caminho_padrao.replace('Ativos.xlsx', 'Ativos_v7.0.xlsx')

# Selecionar arquivo
try:
    EXCEL_PATH = selecionar_arquivo_excel(EXCEL_DIR, EXCEL_PATTERN, caminho_padrao)
    FONTE_DADOS = "EXCEL"
except FileNotFoundError as e:
    print(f"⚠️ {e}")
    FONTE_DADOS = "ERRO"
    raise

print(f"\n📊 Fonte: {FONTE_DADOS}")

SELEÇÃO DE FONTE DE DADOS
✅ Único arquivo encontrado: Rendimentos_Mensais_Ativos_v7.0.xlsx

📊 Fonte: EXCEL


In [12]:
# =============================================================================
# CARREGAMENTO DOS DADOS
# =============================================================================

print("\n📂 Carregando dados do Excel...")
print("-" * 50)

retornos_df, pesos_carteira_raw, relatorio = carregar_dados_excel(
    EXCEL_PATH, 
    min_obs=MIN_OBS, 
    ativos_excluir=ATIVOS_EXCLUIR
)

print(f"\n✅ Ativos carregados: {len(relatorio['abas_processadas'])}")

if relatorio['ativos_excluidos_manual']:
    print(f"🚫 Ativos excluídos manualmente: {relatorio['ativos_excluidos_manual']}")

if relatorio['abas_excluidas']:
    print(f"⚠️ Ativos excluídos por dados insuficientes: {len(relatorio['abas_excluidas'])}")
    for ativo, motivo in relatorio['abas_excluidas'][:5]:
        print(f"   - {ativo}: {motivo}")

print(f"\n📅 Período: {retornos_df.index.min().strftime('%Y-%m')} a {retornos_df.index.max().strftime('%Y-%m')}")
print(f"   Total de meses: {len(retornos_df)}")

ATIVOS_OTIMIZACAO = list(retornos_df.columns)
NUM_ATIVOS = len(ATIVOS_OTIMIZACAO)

print(f"\n🎯 Ativos no universo de otimização: {NUM_ATIVOS}")


📂 Carregando dados do Excel...
--------------------------------------------------

📊 Buscando coluna de pesos...
   ✅ Coluna de pesos encontrada: 'Peso Carteira Atual' (linha 15, col 1)

✅ Ativos carregados: 31
🚫 Ativos excluídos manualmente: ['USDC']
⚠️ Ativos excluídos por dados insuficientes: 3
   - CRA REDE SIM - FEV-2030: 9 meses < 12
   - DEB ENGIE - SET-2030: 8 meses < 12
   - Valora CRI CDI Renda+ FII RL: 1 meses < 12

📅 Período: 2022-01 a 2025-12
   Total de meses: 48

🎯 Ativos no universo de otimização: 31


In [13]:
# =============================================================================
# MAPEAMENTO DE PESOS DA CARTEIRA ATUAL
# =============================================================================

print("\n" + "=" * 60)
print("MAPEAMENTO DE PESOS DA CARTEIRA ATUAL")
print("=" * 60)

if relatorio['status_pesos'] == 'nao_encontrado':
    print("\n⚠️ Coluna de pesos não encontrada no Excel!")
    print("   Continuando sem pesos da carteira atual...")
    pesos_carteira = None
    TEM_CARTEIRA_ATUAL = False

elif pesos_carteira_raw:
    mapeamento = {}
    nao_mapeados = []
    
    for ativo_resumo, peso in pesos_carteira_raw.items():
        if peso == 0:
            continue
        
        # Match exato
        if ativo_resumo in ATIVOS_OTIMIZACAO:
            mapeamento[ativo_resumo] = {'aba': ativo_resumo, 'peso': peso, 'score': 1.0}
            continue
        
        # Fuzzy match
        melhor_match = None
        melhor_score = 0
        for aba in ATIVOS_OTIMIZACAO:
            score = similaridade(ativo_resumo, aba)
            if score > melhor_score:
                melhor_score = score
                melhor_match = aba
        
        if melhor_score >= 0.55:  # Threshold mais baixo
            mapeamento[ativo_resumo] = {'aba': melhor_match, 'peso': peso, 'score': melhor_score}
        else:
            nao_mapeados.append((ativo_resumo, peso, melhor_match, melhor_score))
    
    print(f"\n✅ Ativos mapeados: {len(mapeamento)}")
    
    # Mostrar mapeamentos fuzzy
    fuzzy_maps = [(k, v) for k, v in mapeamento.items() if v['score'] < 1.0]
    if fuzzy_maps:
        print("\n   Mapeamentos por similaridade:")
        for ativo, info in sorted(fuzzy_maps, key=lambda x: -x[1]['peso'])[:10]:
            print(f"   '{ativo[:30]}' → '{info['aba']}' (score: {info['score']:.2f}, peso: {info['peso']:.2%})")
    
    if nao_mapeados:
        peso_nao_mapeado = sum(x[1] for x in nao_mapeados)
        print(f"\n⚠️ Ativos NÃO mapeados: {len(nao_mapeados)} (peso: {peso_nao_mapeado:.2%})")
        for ativo, peso, match, score in sorted(nao_mapeados, key=lambda x: -x[1])[:5]:
            print(f"   '{ativo[:40]}' (melhor: '{match}', score: {score:.2f})")
    
    # Criar vetor de pesos
    pesos_carteira = np.zeros(NUM_ATIVOS)
    for ativo, info in mapeamento.items():
        try:
            idx = ATIVOS_OTIMIZACAO.index(info['aba'])
            pesos_carteira[idx] += info['peso']
        except ValueError:
            pass
    
    soma_pesos = pesos_carteira.sum()
    print(f"\n📊 Soma dos pesos mapeados: {soma_pesos:.2%}")
    
    if soma_pesos > 0 and abs(soma_pesos - 1.0) > 0.01:
        print(f"   Renormalizando para 100%")
        pesos_carteira = pesos_carteira / soma_pesos
    
    TEM_CARTEIRA_ATUAL = soma_pesos > 0
else:
    pesos_carteira = None
    TEM_CARTEIRA_ATUAL = False
    print("\n⚠️ Nenhum peso encontrado.")


MAPEAMENTO DE PESOS DA CARTEIRA ATUAL

✅ Ativos mapeados: 31

   Mapeamentos por similaridade:
   'CRA REDE SIM - FEV/2030' → 'CRA JBS - SET-2038' (score: 0.59, peso: 16.23%)
   'ARX Hedge FIC Incentivado Fina' → 'ARX Hedge FIC Incentivado Finan' (score: 0.95, peso: 7.88%)
   'DEB ENGIE - SET/2030' → 'DEB AEGEA RIO SPE4 - SET-2042' (score: 0.85, peso: 6.23%)
   'XP Debêntures Incentivadas CDI' → 'XP Debêntures Incentivadas CDI ' (score: 0.95, peso: 5.40%)
   'GS Índice de Commodities Long ' → 'COE GS Commodities' (score: 0.85, peso: 0.53%)
   'Plural Yield FIF RF Referencia' → 'Plural Yield FIF RF Referenciad' (score: 0.95, peso: 0.23%)
   'Absolute Atenas Advisory FIC F' → 'Absolute Atenas Advisory FIC FI' (score: 0.95, peso: 0.10%)
   'Trend Investback FIC FIRF Simp' → 'Trend Investback FIC FIRF Simpl' (score: 0.95, peso: 0.08%)
   'Sparta Debêntures Incentivadas' → 'Sparta Debêntures Incentivadas ' (score: 0.90, peso: 0.07%)

⚠️ Ativos NÃO mapeados: 2 (peso: 8.71%)
   'Valora CRI C

---
## 📈 BLOCO 3 - CÁLCULO DE ESTATÍSTICAS

In [17]:
# =============================================================================
# TAXA LIVRE DE RISCO
# =============================================================================

print("\n" + "=" * 60)
print("TAXA LIVRE DE RISCO (CDI)")
print("=" * 60)

# Usar período dos dados
periodo_inicio = retornos_df.index.min().strftime('%Y-%m-%d')
periodo_fim = retornos_df.index.max().strftime('%Y-%m-%d')

RF_ANUAL = obter_taxa_livre_risco(RF_MODO, RF_FALLBACK, RF_MANUAL, periodo_inicio, periodo_fim)

print(f"\n📊 RF anual (CDI): {RF_ANUAL*100:.2f}% a.a.")
print(f"   Modo: {RF_MODO}")
print(f"   Período alinhado: {periodo_inicio} a {periodo_fim}")


TAXA LIVRE DE RISCO (CDI)

📊 RF anual (CDI): 14.90% a.a.
   Modo: atual
   Período alinhado: 2022-01-01 a 2025-12-01


In [18]:
# =============================================================================
# ESTATÍSTICAS DOS ATIVOS
# =============================================================================

print("\n" + "=" * 60)
print("ESTATÍSTICAS DOS ATIVOS")
print("=" * 60)

retornos_esperados, cov_matrix = calcular_estatisticas_portfolio(
    retornos_df, 
    usar_geometrico=USAR_MEDIA_GEOMETRICA, 
    vol_floor_mensal=VOL_FLOOR_MENSAL
)

cov_matrix, foi_corrigida = corrigir_matriz_psd(cov_matrix)
if foi_corrigida:
    print("\n⚠️ Matriz de covariância corrigida para PSD")

corr_matrix = retornos_df.corr()
volatilidades = pd.Series({col: np.sqrt(cov_matrix.loc[col, col]) for col in cov_matrix.columns})

print(f"\n{'Ativo':<30} {'Ret. Anual':<12} {'Vol. Anual':<12} {'Sharpe':<10} {'Meses':<8}")
print("-" * 75)
for ativo in ATIVOS_OTIMIZACAO[:20]:
    ret = retornos_esperados[ativo]
    vol = volatilidades[ativo]
    sharpe = (ret - RF_ANUAL) / vol if vol > 1e-6 else np.nan
    n_meses = relatorio['meses_por_ativo'].get(ativo, 0)
    print(f"{ativo:<30} {ret*100:>9.2f}% {vol*100:>9.2f}% {sharpe:>9.2f} {n_meses:>6}")

if len(ATIVOS_OTIMIZACAO) > 20:
    print(f"... e mais {len(ATIVOS_OTIMIZACAO) - 20} ativos")

print(f"\n📊 Método de retorno: {'Geométrico' if USAR_MEDIA_GEOMETRICA else 'Aritmético'}")


ESTATÍSTICAS DOS ATIVOS
   ⚠️ VOL_FLOOR aplicado em 6 ativos: ['Absolute Atenas Advisory FIC FI', 'BNP Paribas RF FIF RF', 'Daycoval Classic CIC FIRF CP RL', 'Fundo 24 Horas FIRF RL', 'Plural Yield FIF RF Referenciad']...

⚠️ Matriz de covariância corrigida para PSD

Ativo                          Ret. Anual   Vol. Anual   Sharpe     Meses   
---------------------------------------------------------------------------
PETR4                              28.81%     28.43%      0.49     47
ITUB4                              24.07%     24.12%      0.38     47
VALE3                               7.12%     28.00%     -0.28     47
BBAS3                              17.16%     24.84%      0.09     47
GOOGL                              24.21%     29.18%      0.32     47
NVDA                               68.26%     51.90%      1.03     47
NDAQ                               15.15%     24.08%      0.01     47
META                               21.45%     44.76%      0.15     47
AMZN              

---
## ⚙️ BLOCO 4 - OTIMIZAÇÃO DE PORTFÓLIO

In [19]:
# =============================================================================
# OTIMIZAÇÃO: GMV, MAX SHARPE E FRONTEIRA
# =============================================================================

print("\n" + "=" * 60)
print("OTIMIZAÇÃO DE PORTFÓLIO")
print("=" * 60)

ret_array = retornos_esperados.values
cov_array = cov_matrix.values

print("\n🔍 Otimizando GMV...")
pesos_gmv = otimizar_min_volatilidade(ret_array, cov_array, PESO_MIN_ATIVO, PESO_MAX_ATIVO)
ret_gmv, vol_gmv, sharpe_gmv = calcular_metricas_portfolio(pesos_gmv, ret_array, cov_array, RF_ANUAL)

print("🔍 Otimizando Máximo Sharpe...")
pesos_max_sharpe = otimizar_max_sharpe(ret_array, cov_array, RF_ANUAL, PESO_MIN_ATIVO, PESO_MAX_ATIVO)
ret_max_sharpe, vol_max_sharpe, sharpe_max_sharpe = calcular_metricas_portfolio(pesos_max_sharpe, ret_array, cov_array, RF_ANUAL)

print("🔍 Calculando Fronteira Eficiente...")
fronteira_vols, fronteira_rets = calcular_fronteira_eficiente(
    ret_array, cov_array, RF_ANUAL, 
    n_pontos=N_PONTOS_FRONTEIRA,
    peso_min=PESO_MIN_ATIVO, peso_max=PESO_MAX_ATIVO
)
print(f"   {len(fronteira_vols)} pontos calculados")

if TEM_CARTEIRA_ATUAL:
    ret_atual, vol_atual, sharpe_atual = calcular_metricas_portfolio(pesos_carteira, ret_array, cov_array, RF_ANUAL)

print("\n" + "=" * 60)
print("RESULTADOS")
print("=" * 60)
print(f"\n{'Carteira':<20} {'Retorno':<15} {'Volatilidade':<15} {'Sharpe':<10}")
print("-" * 60)
print(f"{'GMV':<20} {ret_gmv*100:>12.2f}% {vol_gmv*100:>12.2f}% {sharpe_gmv:>10.3f}")
print(f"{'Max Sharpe':<20} {ret_max_sharpe*100:>12.2f}% {vol_max_sharpe*100:>12.2f}% {sharpe_max_sharpe:>10.3f}")
if TEM_CARTEIRA_ATUAL:
    print(f"{'Carteira Atual ⭐':<20} {ret_atual*100:>12.2f}% {vol_atual*100:>12.2f}% {sharpe_atual:>10.3f}")


OTIMIZAÇÃO DE PORTFÓLIO

🔍 Otimizando GMV...
🔍 Otimizando Máximo Sharpe...
🔍 Calculando Fronteira Eficiente...
   60 pontos calculados

RESULTADOS

Carteira             Retorno         Volatilidade    Sharpe    
------------------------------------------------------------
GMV                         12.70%         0.82%     -2.694
Max Sharpe                  44.54%        26.01%      1.140
Carteira Atual ⭐             9.22%        10.46%     -0.544


---
## 🎲 BLOCO 5 - SIMULAÇÃO MONTE CARLO

In [20]:
# =============================================================================
# SIMULAÇÃO MONTE CARLO (MELHORADA)
# =============================================================================

print("\n" + "=" * 60)
print(f"SIMULAÇÃO MONTE CARLO ({NUM_PORTFOLIOS:,} portfólios)")
print("=" * 60)

ret_sim, vol_sim, sharpe_sim, pesos_sim = simular_portfolios_monte_carlo_v2(
    ret_array, cov_array, RF_ANUAL,
    n_portfolios=NUM_PORTFOLIOS,
    seed=RANDOM_SEED,
    alpha_conc=MC_ALPHA_CONC,
    alpha_div=MC_ALPHA_DIV,
    frac_cantos=MC_FRAC_CANTOS,
    frac_sparse=MC_FRAC_SPARSE
)

print(f"\n✅ Simulação concluída!")
print(f"   Retorno: min={ret_sim.min()*100:.2f}%, max={ret_sim.max()*100:.2f}%")
print(f"   Volatilidade: min={vol_sim.min()*100:.2f}%, max={vol_sim.max()*100:.2f}%")
print(f"   Sharpe: min={np.nanmin(sharpe_sim):.3f}, max={np.nanmax(sharpe_sim):.3f}")


SIMULAÇÃO MONTE CARLO (80,000 portfólios)

✅ Simulação concluída!
   Retorno: min=-0.97%, max=67.99%
   Volatilidade: min=1.11%, max=121.83%
   Sharpe: min=-2.615, max=1.113


---
## 📊 BLOCO 6 - VISUALIZAÇÕES

In [21]:
# =============================================================================
# GRÁFICO: FRONTEIRA EFICIENTE
# =============================================================================

carteiras_especiais = {
    'GMV': {
        'retorno': ret_gmv, 'volatilidade': vol_gmv, 'sharpe': sharpe_gmv,
        'pesos': pesos_gmv, 'nomes': ATIVOS_OTIMIZACAO
    },
    'Max Sharpe': {
        'retorno': ret_max_sharpe, 'volatilidade': vol_max_sharpe, 'sharpe': sharpe_max_sharpe,
        'pesos': pesos_max_sharpe, 'nomes': ATIVOS_OTIMIZACAO
    }
}

if TEM_CARTEIRA_ATUAL:
    carteiras_especiais['Carteira Atual'] = {
        'retorno': ret_atual, 'volatilidade': vol_atual, 'sharpe': sharpe_atual,
        'pesos': pesos_carteira, 'nomes': ATIVOS_OTIMIZACAO
    }

fig_frontier = criar_grafico_fronteira_completo(
    ret_sim, vol_sim, sharpe_sim,
    fronteira_vols, fronteira_rets,
    carteiras_especiais, RF_ANUAL,
    titulo="Fronteira Eficiente - PyPortfolio v2.4"
)
fig_frontier.show()

In [ ]:
# =============================================================================
# GRÁFICO: MATRIZ DE CORRELAÇÃO
# =============================================================================

fig_corr = criar_heatmap_correlacao(corr_matrix, "Matriz de Correlação - Retornos Mensais")
fig_corr.show()

In [ ]:
# =============================================================================
# GRÁFICO: COMPARATIVO DE PESOS
# =============================================================================

if TEM_CARTEIRA_ATUAL:
    fig_pesos = criar_grafico_barras_pesos(carteiras_especiais, ATIVOS_OTIMIZACAO, top_n=TOP_N_PESOS)
    fig_pesos.show()

---
## 📋 BLOCO 7 - RESUMO FINAL

In [ ]:
# =============================================================================
# RESUMO FINAL
# =============================================================================

print("\n" + "=" * 70)
print("RESUMO DA ANÁLISE - PyPortfolio v2.4")
print("=" * 70)

print(f"\n📊 CONFIGURAÇÃO")
print(f"   Arquivo: {Path(EXCEL_PATH).name}")
print(f"   Período: {retornos_df.index.min().strftime('%Y-%m')} a {retornos_df.index.max().strftime('%Y-%m')}")
print(f"   Ativos no universo: {NUM_ATIVOS}")
print(f"   Ativos excluídos: {ATIVOS_EXCLUIR}")
print(f"   VOL_FLOOR: {VOL_FLOOR_MENSAL*100:.1f}% mensal")
print(f"   Taxa livre de risco (CDI): {RF_ANUAL*100:.2f}% a.a. (modo: {RF_MODO})")

print(f"\n📈 CARTEIRAS OTIMIZADAS")
print(f"   {'Carteira':<20} {'Retorno':<12} {'Vol':<12} {'Sharpe':<10}")
print(f"   {'-'*54}")
print(f"   {'GMV':<20} {ret_gmv*100:>9.2f}%  {vol_gmv*100:>9.2f}%  {sharpe_gmv:>8.3f}")
print(f"   {'Max Sharpe':<20} {ret_max_sharpe*100:>9.2f}%  {vol_max_sharpe*100:>9.2f}%  {sharpe_max_sharpe:>8.3f}")
if TEM_CARTEIRA_ATUAL:
    print(f"   {'Carteira Atual ⭐':<20} {ret_atual*100:>9.2f}%  {vol_atual*100:>9.2f}%  {sharpe_atual:>8.3f}")

if TEM_CARTEIRA_ATUAL:
    print(f"\n💡 DIAGNÓSTICO DA CARTEIRA ATUAL")
    diff_sharpe = sharpe_max_sharpe - sharpe_atual
    diff_vol = vol_atual - vol_gmv
    
    if diff_sharpe > 0.1:
        print(f"   ⚠️  Sharpe atual está {diff_sharpe:.2f} abaixo do ótimo")
    else:
        print(f"   ✅ Sharpe atual está próximo do ótimo")
    
    if diff_vol > 0.02:
        print(f"   ⚠️  Volatilidade {diff_vol*100:.1f}pp acima do GMV")
    else:
        print(f"   ✅ Volatilidade próxima do GMV")

# Top pesos das carteiras
print(f"\n📊 TOP 10 PESOS")
print("-" * 50)
print(f"{'GMV:':<50}")
for i in np.argsort(pesos_gmv)[::-1][:10]:
    if pesos_gmv[i] > 0.01:
        print(f"   {ATIVOS_OTIMIZACAO[i]:<30} {pesos_gmv[i]*100:>8.2f}%")

print(f"\n{'Max Sharpe:':<50}")
for i in np.argsort(pesos_max_sharpe)[::-1][:10]:
    if pesos_max_sharpe[i] > 0.01:
        print(f"   {ATIVOS_OTIMIZACAO[i]:<30} {pesos_max_sharpe[i]*100:>8.2f}%")

print("\n" + "=" * 70)
print("✅ Análise concluída com sucesso!")
print("=" * 70)

---

## 📝 Notas e Observações

### Sobre o USDC
O USDC foi **removido do universo de otimização** porque:
- É uma stablecoin com volatilidade próxima de zero (~0.3% a.a.)
- Retorno ~0% (variações apenas de peg)
- Distorce a carteira GMV (puxa para vol artificial de ~0%)
- Tratamento recomendado: alocar separadamente como "caixa"

### Sobre Sharpe Negativo
Sharpe negativo é **esperado** quando:
- CDI (RF) atual está alto (~14-15% a.a.)
- Retorno do ativo é menor que CDI
- Não significa que o ativo é "ruim", apenas que não supera a taxa livre de risco

### Melhorias Implementadas (v2.4)
1. ✅ Detecção automática de arquivos Excel
2. ✅ Leitura robusta da coluna de pesos (aceita "Carteria")
3. ✅ USDC removido da otimização
4. ✅ Curva da fronteira eficiente calculada
5. ✅ Monte Carlo com cantos e sparse sampling
6. ✅ VOL_FLOOR na matriz de covariância
7. ✅ Fuzzy matching melhorado (hífen/barra tolerante)
8. ✅ Relatório de qualidade de dados

## LIMITAÇÕES CONHECIDAS (v2.4)

1. **yfinance não implementado** - Apenas Excel como fonte primária nesta versão
2. **Sem restrições por classe** - Long-only genérico apenas
3. **Sem Black-Litterman** - Apenas Markowitz clássico
4. **Fuzzy matching simples** - Pode não mapear nomes muito diferentes
5. **Sem backtesting** - Análise apenas in-sample
6. **Exclusão de abas dosativos antes de mapear** - Ativos que tiveram suas abas excluidas e estão na aba resumo acabam sendo mapeados erroneamente. Deveriam ser exclidos também